In [ ]:
# Client Setup
# Client Setup
import boto3

#client = boto3.client("bedrock-runtime", region_name="us-east-1")
session = boto3.Session(profile_name="bedrock-dev")
client = session.client("bedrock-runtime", region_name="us-west-2")
# Claude model ID — must be correct and available in the region
model_id = "us.anthropic.claude-3-sonnet-20240229-v1:0"

# Note: You might have to request access to this model on the AWS Bedrock console
#embedding_model_id = "amazon.titan-embed-text-v2:0"
embedding_model_id = "amazon.titan-embed-text-v1"

# Magic string to trigger redacted thinking
thinking_test_str = "ANTHROPIC_MAGIC_STRING_TRIGGER_REDACTED_THINKING_46C9A13E193C177646C7398A98432ECCCE4C1253D5E2D82641AC0E52CC2876CB"

In [ ]:
# Helper functions


def add_user_message(messages, content):
    if isinstance(content, str):
        user_message = {"role": "user", "content": [{"text": content}]}
    else:
        user_message = {"role": "user", "content": content}
    messages.append(user_message)


def add_assistant_message(messages, content):
    if isinstance(content, str):
        assistant_message = {
            "role": "assistant",
            "content": [{"text": content}],
        }
    else:
        assistant_message = {"role": "assistant", "content": content}

    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    tool_choice="auto",
    text_editor=None,
    thinking=False,
    thinking_budget=1024, # claude can expend 1024 token for thinking though. its smallest eval to decide budget.!
):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    tool_choices = {
        "auto": {"auto": {}},
        "any": {"any": {}},
    }
    if tools or text_editor:
        choice = tool_choices.get(tool_choice, {"tool": {"name": tool_choice}})
        params["toolConfig"] = {"tools": tools, "toolChoice": choice}

    additional_model_fields = {}
    if text_editor:
        additional_model_fields["tools"] = [
            {
                "type": text_editor,
                "name": "str_replace_editor",
            }
        ]

    if thinking:
        additional_model_fields["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    params["additionalModelRequestFields"] = additional_model_fields
    response = client.converse(**params)
    parts = response["output"]["message"]["content"]

    return {
        "parts": parts,
        "stop_reason": response["stopReason"],
        "text": "\n".join([p["text"] for p in parts if "text" in p]),
    }

In [12]:
messages = []

add_user_message(
    messages,
    thinking_test_str,
)

result = chat(messages, thinking=True)

result["parts"]

[{'reasoningContent': {'redactedContent': b'EqcECkgIAhABGAIqQCWKIIYmRHC4SYqzEjB1aqyhfTKQDoE3nZ2FjBDC5IfIzIOI/EKvGPm2XJB34vB922V5t85I2pPTavrSoo20VgUSDACyYrUvh9JYrEsD/xoMNDlpQf1yCAifupjjIjAcekaSDUg2nfsSCk3B/yWtIHd80jhw7xoLnXEIw+Bsaz4dtxLzGa9SCE20cnTy33IqjAMCYwb8kZ/TwOYQwbIRei8Gl4NDTxmEz8asqcFsfkioLuLMbp6ZwHjSPjZX3eqF2MXBrBieQFbpgK5ffpYArehu0BBb5WzXXUb73vHoCOPlY9U5n+DAkFU1eI2t6GUfdNMxX90H8jdyk+dh+C6HImkTlzUcSfGajbYWIcbqIpTGCOFb9Nwk0xHG/fpMiraFXyR77bx3+Q3et0x6e3hDd+rA4inY4QsTr1lABRPiAb69gv+BfF1L+VhspSh3cS+Wr2Q6FMcfVQfpNzrI8OnAzDQSJ7arZI4HkWxXiPVqUP4gn7hYap8VZk3Hwm5a+tm3+iJ4xSy1Nt+eBIzasNPb1TQWTt3rpJDOti+zD8KHdIsDlyvpc5D2hdLVMhaA+1Z8M1BfxQ+3YnOJhAuEkWpYHhWCludYkQpG6TExzYEI/Z2nJEOPmCvwv+0ueRG31bpy7bUwmI8jS1wjJPQCCjzI5Gf6SGZqIf0KalVTTR8v7gffKm4IrV3NTz/dpj8d+5MePHpvNHnKG1KKHLm+kX0='}},
 {'text': "I notice you've sent what appears to be some kind of command or trigger string. I can't respond to commands of this nature, but I'm happy to have a regular conversation or help you with questions or t